In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Conv1D, MaxPooling1D, Flatten
from google.colab import drive

In [2]:
drive.mount('/content/drive')
file_path = '/content/drive/MyDrive/Data1.xlsx'
df = pd.read_excel(file_path)
df = df.drop(["No."],axis = 1)
display(df)

Mounted at /content/drive


,Tradevalue,Interest,CPI,GDPTW,GDPID,Exchangerate_TW,Exchangerate_ID
0,1043.44,4.543333,81.240000,81139,135355.256125,30.74,7483.333333
1,1254.46,4.670000,81.673333,81801,119275.774773,30.68,8443.333333
2,1236.55,4.750000,81.866667,83514,118987.521626,31.07,8670.000000
3,1214.16,4.710000,82.910000,84271,111509.035986,32.45,9517.500000
4,948.45,4.380000,81.713333,77571,106490.601314,32.53,9895.000000
...,...,...,...,...,...,...,...
91,3287.84,1.670000,103.723333,184142,190205.871091,31.37,15670.000000
92,2631.34,1.793333,104.363333,180361,199728.169480,30.40,15105.000000
93,2623.53,1.880000,105.103333,185510,204896.275645,30.71,14915.333333
94,2743.45,1.880000,105.876667,191900,202499.100288,31.69,15282.666667


In [3]:
# prompt: normalize the contents from dataframe above using min max scaling with range from 0 to 1

# Assuming 'df' is your DataFrame and you want to normalize all numerical columns
scaler = MinMaxScaler(feature_range=(0, 1))

# Select only numerical columns for scaling
numerical_cols = df.select_dtypes(include=np.number).columns
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

display(df)

,Tradevalue,Interest,CPI,GDPTW,GDPID,Exchangerate_TW,Exchangerate_ID
0,0.068647,0.942910,0.000654,0.072240,0.351938,0.402778,0.000000
1,0.130893,0.977901,0.017659,0.077154,0.216892,0.394444,0.117264
2,0.125610,1.000000,0.025245,0.089870,0.214471,0.448611,0.144951
3,0.119006,0.988950,0.066187,0.095489,0.151661,0.640278,0.248473
4,0.040627,0.897790,0.019228,0.045755,0.109513,0.651389,0.294585
...,...,...,...,...,...,...,...
91,0.730694,0.149171,0.882930,0.836828,0.812609,0.490278,1.000000
92,0.537042,0.183241,0.908044,0.808762,0.892584,0.355556,0.930985
93,0.534738,0.207182,0.937083,0.846983,0.935989,0.398611,0.907818
94,0.570112,0.207182,0.967430,0.894416,0.915856,0.534722,0.952687


In [4]:
df["Tradevalue"] = df["Tradevalue"].shift(-1)
df.dropna(inplace=True)

In [5]:
X = df.drop("Tradevalue",axis=1)
y = df["Tradevalue"]

In [6]:
train_size = int(len(df) * 0.9)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

In [7]:
# Define time_steps
time_steps = 3  # Using more than 1 to capture temporal dependencies

# Function to create sequences
def create_sequences(X, y, time_steps):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        Xs.append(X[i:(i + time_steps)])
        ys.append(y[i + time_steps])
    return np.array(Xs), np.array(ys)

# Create sequences
X_sequences, y_sequences = create_sequences(X, y, time_steps)

# Train-test split
train_size = int(len(X_sequences) * 0.9)
X_train, X_test = X_sequences[:train_size], X_sequences[train_size:]
y_train, y_test = y_sequences[:train_size], y_sequences[train_size:]

# Define the model
model = Sequential()

# Conv1D layers require input sequences longer than the kernel_size
model.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(MaxPooling1D(pool_size=1))
model.add(Dropout(0.2))

# LSTM layers
model.add(LSTM(128, activation='relu', return_sequences=True))
model.add(Dropout(0.3))

model.add(LSTM(64, activation='relu'))
model.add(Dropout(0.3))

# Dense layers
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(32, activation='relu'))
model.add(Dense(1))

# Compile the model with the correct loss function for regression
model.compile(optimizer='adam', loss='mean_absolute_error')

# Train the model
model.fit(X_train, y_train, epochs=1000, batch_size=8, validation_data=(X_test, y_test), verbose=1)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 4s 32ms/step - loss: 0.3541 - val_loss: 0.5439
Epoch 2/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.2481 - val_loss: 0.1769
Epoch 3/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.1855 - val_loss: 0.2356
Epoch 4/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.1682 - val_loss: 0.3049
Epoch 5/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.1383 - val_loss: 0.1575
Epoch 6/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.1405 - val_loss: 0.1463
Epoch 7/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.1491 - val_loss: 0.2645
Epoch 8/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.1171 - val_loss: 0.2094
Epoch 9/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0953 - val_loss: 0.2697
Epoch 10/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.1117 - val_loss: 0.2309
Epoch 11/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0912 - val_loss: 0.3327
Epoch 12/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/ste

In [8]:
# Forecasting the next 10 periods
last_sequence = X_test[-1]
predictions = []

In [9]:
for _ in range(10):
    pred = model.predict(last_sequence[np.newaxis, :, :])
    predictions.append(pred[0, 0])
    last_sequence = np.vstack((last_sequence[1:], np.append(pred, last_sequence[0, :-1])))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 198ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step


In [10]:
print(last_sequence)

[[0.28663558 0.52209866 0.35544518 0.18324125 0.90804447 0.80876207]
 [0.22458649 0.47036073 0.48180518 0.20718232 0.93708306 0.84698293]
 [0.2164458  0.46646139 0.58380389 0.39924088 0.14917127 0.88293002]]


In [12]:
np.hstack((
    np.tile(last_sequence[0, :-1], (len(predictions), 1)),  # Match rows to predictions
    np.array(predictions).reshape(-1, 1)
))

array([[0.28663558, 0.52209866, 0.35544518, 0.18324125, 0.90804447,
        0.39924088],
       [0.28663558, 0.52209866, 0.35544518, 0.18324125, 0.90804447,
        0.35544518],
       [0.28663558, 0.52209866, 0.35544518, 0.18324125, 0.90804447,
        0.48180518],
       [0.28663558, 0.52209866, 0.35544518, 0.18324125, 0.90804447,
        0.58380389],
       [0.28663558, 0.52209866, 0.35544518, 0.18324125, 0.90804447,
        0.52209866],
       [0.28663558, 0.52209866, 0.35544518, 0.18324125, 0.90804447,
        0.47036073],
       [0.28663558, 0.52209866, 0.35544518, 0.18324125, 0.90804447,
        0.46646139],
       [0.28663558, 0.52209866, 0.35544518, 0.18324125, 0.90804447,
        0.28663558],
       [0.28663558, 0.52209866, 0.35544518, 0.18324125, 0.90804447,
        0.22458649],
       [0.28663558, 0.52209866, 0.35544518, 0.18324125, 0.90804447,
        0.2164458 ]])

In [13]:
predictions = predictions[:10]  # Truncate to 10 elements (if longer)
# or
predictions = np.pad(predictions, (0, 10 - len(predictions)), constant_values=0)  # Pad with zeros (if shorter)

In [14]:
print("Shape of last_sequence[0, :-1]:", last_sequence[0, :-1].shape)
print("Shape of predictions:", np.array(predictions).shape)

Shape of last_sequence[0, :-1]: (5,)
Shape of predictions: (10,)


In [15]:
# Inverse transform the predictions
predictions_rescaled = scaler.inverse_transform(
    np.hstack((np.tile(last_sequence[0, :-1], (10, 1)), np.array(predictions).reshape(-1, 1)))
)[:, -1]

ValueError: operands could not be broadcast together with shapes (10,6) (7,) (10,6) 